In [1]:
'''
If you are running locally then 
1. reboot your local machine
2. create an environment called 'colab' using anaconda prompt
if you have a gpu
conda create -n colab python tensorflow-gpu 
if not 
conda create -n colab python tensorflow
3. to install jupyter notebook
conda install jupyter notebook
4. to go to the 'colab' environment
activate colab
5. change file path to locate this notebook and then type 'jupyter notebook'

If you use colab
1. save the data file in your google drive
2. goto colab and start running the code
'''

"\nIf you are running locally then \n1. reboot your local machine\n2. create an environment called 'colab' using anaconda prompt\nif you have a gpu\nconda create -n colab python tensorflow-gpu \nif not \nconda create -n colab python tensorflow\n3. to install jupyter notebook\nconda install jupyter notebook\n4. to go to the 'colab' environment\nactivate colab\n5. change file path to locate this notebook and then type 'jupyter notebook'\n\nIf you use colab\n1. save the data file in your google drive\n2. goto colab and start running the code\n"

In [2]:
#install prominent libraries with specific versions

#!pip install tensorflow==1.15.0
#!pip install keras==2.2.4-tf
#!pip install pandas==0.25.1
#!pip install sklearn==0.21.3
#!pip install matplotlib==3.2.1
#!pip install hyperas
#!pip install hyperopt

In [3]:
from __future__ import print_function                                                                                                                                                                                                                                                                                                                              # from tensorflow.contrib.rnn import *import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, GRU, Input, Activation, Flatten, BatchNormalization, Reshape,Concatenate
from keras.callbacks import ModelCheckpoint, TensorBoard, History
from keras.models import Model, load_model
from keras.layers.advanced_activations import LeakyReLU
from keras import regularizers
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from keras.wrappers.scikit_learn import KerasClassifier
import matplotlib.pyplot as plt
from matplotlib import style
from keras.optimizers import SGD, RMSprop, Adam, Adadelta
from keras.utils import np_utils
from sklearn.preprocessing import*
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers import BatchNormalization
from keras.optimizers import SGD, RMSprop, Adam, Adadelta
from keras.utils import np_utils
import pandas as pd
from keras import regularizers
from keras.callbacks import ModelCheckpoint
from collections import Counter
import operator
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
import itertools
import hyperas
import hyperopt
from hyperas.distributions import choice, uniform
from hyperas import optim
from hyperopt import Trials, STATUS_OK, tpe, rand
from keras.layers import Conv1D, MaxPooling1D, ZeroPadding1D

Using TensorFlow backend.


In [4]:
#Get library versions
print("tensorflow.__version__ = ", tf.__version__)
print("keras.__version__ = ", keras.__version__)
import sklearn 
print("sklearn.__version__ = ", sklearn.__version__)
print("numpy.__version__ = ", np.__version__)
print("pandas.__version__ = ", pd.__version__)
import matplotlib
print("matplotlib.__version__ = ", matplotlib.__version__)

tensorflow.__version__ =  1.15.0
keras.__version__ =  2.2.4-tf
sklearn.__version__ =  0.21.3
numpy.__version__ =  1.16.5
pandas.__version__ =  0.25.1
matplotlib.__version__ =  3.2.1


In [5]:
from numpy.random import seed
seed(56)
from tensorflow import set_random_seed
set_random_seed(56)

In [6]:
# create these folders if they do not exist
def build_path(dirName):
    try:
        os.makedirs(dirName)    
        print("Directory " , dirName ,  " Created ")
    except:
        print("Directory " , dirName ,  " already exists")  

In [7]:
# to check if GPU is getting used locally.....you need to see CPU as well as GPU in the output
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14223989251996743807
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7022277428
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14751148181496636411
physical_device_desc: "device: 0, name: GeForce RTX 2070 SUPER, pci bus id: 0000:06:00.0, compute capability: 7.5"
]


In [8]:
def data():
  i = 7 # the label target - number of days to predict from the input date
  p = 7 #Number of days for target calculated in the data set
  batch_size=32
  CLASSES = 2
  time_steps = 7
  ticker='^GSPC'
    
  # read data
  df=pd.read_csv('../^GSPC_7_days_0_return_dtw.csv', index_col = 0, parse_dates = True)
    
  #add additional rolling mean data  
  rm_window =30
  rolling_mean = []
  for a in range(2,rm_window+1):
    df[ticker+'rm_'+str(a)] = df[ticker].rolling(window=rm_window,center=False).mean()
    rolling_mean.append(ticker+'rm_'+str(a))
    
  # create label
  targets=pd.DataFrame([])
  for j in range (1, p+1):
    targets=targets.append(df[ticker+'_{}d_target'.format(j)])
    targets=targets.append(df[ticker+'_{}d'.format(j)])
  targets=targets.T
  df=df.drop(targets.columns, axis=1)
  df=df[rm_window:-i]
  targets=targets[rm_window:-i]
  y=targets['^GSPC_{}d_target'.format(i)]

  #check for NaN and remove
  df.isna().mean().sum()
  y.isna().mean().sum()
  remove_list=[]
  for i in df.isnull().any().iteritems():
    if i[1] == True:
      remove_list.append(i[0])
  df=df.drop(remove_list, axis=1)
  df.isnull().any().mean()
 
  # add percent change
  df=df.pct_change()
  df=df.replace([np.inf, -np.inf],np.nan) 
  df.fillna(0, inplace=True)
  df.isnull().any().mean()
    
  # apply preprocessing 
  x_scaler=RobustScaler()
  x = x_scaler.fit_transform(df)
  # x_pred = x_scaler.fit_transform(x_pred)
  del df
  y=y.values
    
  # apply time steps
  def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
      v = X[i:(i + time_steps)]
      Xs.append(v)
      ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)
  x, y = create_dataset(x, y, time_steps)

  # create train and test dataset
  x_train, x_test, y_train, y_test=train_test_split(x,y, train_size=0.7, random_state=54)
  x_train = x_train.astype('float32')
  x_test = x_test.astype('float32')
  y_train = y_train.astype('float32')
  y_test = y_test.astype('float32')
    
  # adjustment for batch_size
  train_start = x_train.shape[0]%batch_size
  test_start = x_test.shape[0]%batch_size
  x_train = x_train[train_start:]
  y_train = y_train[train_start:]
  x_test = x_test[test_start:]
  y_test = y_test[test_start:]

  return x_train, x_test, y_train, y_test, batch_size

In [9]:
def create_cnn_dense_model(x_train, x_test, y_train, y_test, batch_size):
    # define two sets of inputs(it can be the same or different)
    inputA = Input(batch_shape=(batch_size, x_train.shape[1], x_train.shape[2]), name='input_1')
    inputB = Input(batch_shape=(batch_size, x_train.shape[1], x_train.shape[2]), name='input_2')

    # the first branch operates on the first input
    x1 = Conv1D(filters = {{choice([32, 64, 128])}},name='x1', kernel_size=2, activation='relu')(inputA)
    x2 = MaxPooling1D(pool_size=2, name='x2')(x1)
    x3 = Conv1D(filters ={{choice([16, 32, 64])}}, name='x3', kernel_size=2, activation='relu')(x2)
    x4 = MaxPooling1D(pool_size=2, name='x4')(x3)
    x5 = Flatten(name='x5')(x4)
    x6 = Dense(16,name='x6', activation='relu')(x5)
    x = Model(inputA, x6, name='x')

    # the second branch opreates on the second input
    y1 = Dense(units = {{choice([32, 64, 128])}}, name='y1', activation="relu")(inputB)
    y2 = Dropout({{uniform(0, 1)}})(y1)
    y3 = Dense(units = {{choice([16, 32, 64])}}, name='y3', activation='relu')(y2)
    y4 = Flatten(name='y4')(y3)
    y5 = Dense(2, name='y5', activation='relu')(y4)
    y = Model(inputB, y5,name='y')

    # combine the output of the two branches
    combined = Concatenate(name='x_y_output_combined')([x.output, y.output])
    z1 = Dropout({{uniform(0, 1)}})(combined)

    # combined outputs
    z = Dense(2, name='z', activation="softmax")(z1)

    # our model will accept the inputs of the two branches and then output buy or sell
    model = Model(inputs=[inputA, inputB], outputs=z)
    
    
    model.summary()
    model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'], optimizer = 'adam')
   
    result = model.fit([x_train, x_train], y_train,  batch_size=32, validation_data=([x_test, x_test], y_test), 
                        epochs=100, verbose=0)
    
    
    
    validation_acc = np.amax(result.history['val_accuracy']) 

    print('Best validation acc of epoch:', result.history['val_accuracy'])
    print('Train acc of epoch:', result.history['accuracy'])
    return {'loss': -validation_acc, 'status': STATUS_OK, 'model': model}

In [10]:
if __name__ == '__main__':
    x_train, x_test, y_train, y_test, _  = data()
    print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)
    
    best_run, best_model = optim.minimize(model=create_cnn_dense_model, data=data ,algo=tpe.suggest, max_evals=5,trials=Trials(), notebook_name='1.1 CNN_Dense_multi_input')
    print("Evalutation of best performing model:")
   
    print(best_model.evaluate([x_test, x_test], y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)

(8832, 7, 2782) (8832,) (3776, 7, 2782) (3776,)
>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import os
except:
    pass

try:
    import tensorflow as tf
except:
    pass

try:
    from tensorflow import keras
except:
    pass

try:
    from keras.models import Sequential
except:
    pass

try:
    from keras.layers import Dense, Dropout, LSTM, GRU, Input, Activation, Flatten, BatchNormalization, Reshape, Concatenate
except:
    pass

try:
    from keras.callbacks import ModelCheckpoint, TensorBoard, History
except:
    pass

try:
    from keras.models import Model, load_model
except:
    pass

try:
    from keras.layers.advanced_activations import LeakyReLU
except:
    pass

try:
    from keras import regularizers
except:
    pass

try:
    from sklearn.decomposition import PCA
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.model_selecti

  48: 
  0%|                                                                            | 0/5 [00:00<?, ?trial/s, best loss=?]WARNING:tensorflow:From C:\Users\aitra\Anaconda3\lib\site-packages\tensorflow_core\python\ops\resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Model: "model_1"                                                                                                       
__________________________________________________________________________________________________                     
Layer (type)                    Output Shape         Param #     Connected to                                          
input_1 (InputLayer)            (32, 7, 2782)        0                                                                 
___________________________

[0.4780190587043762, 0.5095338821411133, 0.5561440587043762, 0.5582627058029175, 0.5511122941970825, 0.5558792352676392, 0.5574682354927063, 0.5566737055778503, 0.5495232939720154, 0.5566737055778503, 0.5561440587043762, 0.5561440587043762, 0.5561440587043762, 0.5579978823661804, 0.5564088821411133, 0.5564088821411133, 0.555084764957428, 0.5577330589294434, 0.5569385886192322, 0.5545550584793091, 0.5548199415206909, 0.5566737055778503, 0.5572034120559692, 0.5564088821411133, 0.5561440587043762, 0.5558792352676392, 0.5545550584793091, 0.5561440587043762, 0.5564088821411133, 0.5566737055778503, 0.5569385886192322, 0.5556144118309021, 0.5556144118309021, 0.5545550584793091, 0.5564088821411133, 0.5579978823661804, 0.5564088821411133, 0.5564088821411133, 0.5561440587043762, 0.5566737055778503, 0.5564088821411133, 0.5564088821411133, 0.5558792352676392, 0.5558792352676392, 0.555349588394165, 0.5569385886192322, 0.5577330589294434, 0.5564088821411133, 0.5566737055778503, 0.5566737055778503, 0

Total params: 539,240                                                                                                  
Trainable params: 539,240                                                                                              
Non-trainable params: 0                                                                                                
__________________________________________________________________________________________________                     
Best validation acc of epoch:                                                                                          
[0.5283368825912476, 0.5529661178588867, 0.554290235042572, 0.5564088821411133, 0.5572034120559692, 0.5439618825912476, 0.5585275292396545, 0.5505826473236084, 0.554290235042572, 0.5564088821411133, 0.554025411605835, 0.5556144118309021, 0.5529661178588867, 0.5564088821411133, 0.5561440587043762, 0.5558792352676392, 0.5564088821411133, 0.5564088821411133, 0.5561440587043762, 0.5572034120559692, 0.

dropout_6 (Dropout)             (32, 18)             0           x_y_output_combined[0][0]                             
__________________________________________________________________________________________________                     
z (Dense)                       (32, 2)              38          dropout_6[0][0]                                       
Total params: 544,120                                                                                                  
Trainable params: 544,120                                                                                              
Non-trainable params: 0                                                                                                
__________________________________________________________________________________________________                     
Best validation acc of epoch:                                                                                          
[0.5166842937469482, 0.5450211763381958,

__________________________________________________________________________________________________                     
dropout_8 (Dropout)             (32, 18)             0           x_y_output_combined[0][0]                             
__________________________________________________________________________________________________                     
z (Dense)                       (32, 2)              38          dropout_8[0][0]                                       
Total params: 541,496                                                                                                  
Trainable params: 541,496                                                                                              
Non-trainable params: 0                                                                                                
__________________________________________________________________________________________________                     
Best validation acc of epoch:           

                                                                 y5[0][0]                                              
__________________________________________________________________________________________________                     
dropout_10 (Dropout)            (32, 18)             0           x_y_output_combined[0][0]                             
__________________________________________________________________________________________________                     
z (Dense)                       (32, 2)              38          dropout_10[0][0]                                      
Total params: 724,008                                                                                                  
Trainable params: 724,008                                                                                              
Non-trainable params: 0                                                                                                
________________________________________